In [ ]:
# Import necessary libraries
import torch
from diffusers import AnimateDiffPipeline, MotionAdapter, DDIMScheduler, EulerDiscreteScheduler
from diffusers.utils import export_to_gif
from huggingface_hub import hf_hub_download
from safetensors.torch import load_file
from IPython.display import Image, display

# Common parameters
device = "cuda"
dtype = torch.float16
base = "emilianJR/epiCRealism"  # Choose your favorite base model.
prompt = "A brave knight rescuing a princess from a dragon's lair."
negative_prompt = "bad quality, worse quality"
num_frames = 24
guidance_scale = 5.0  # Adjusted guidance scale for better quality
num_inference_steps = 100  # Increased number of inference steps for better quality
seed = 42  # Use the same seed for reproducibility

# Function to initialize pipeline
def initialize_pipeline(repo, ckpt, base, device, dtype, use_checkpoint=True, scheduler_type="DDIM"):
    adapter = MotionAdapter().to(device, dtype)
    if use_checkpoint:
        adapter.load_state_dict(load_file(hf_hub_download(repo, ckpt), device=device))
    else:
        adapter = MotionAdapter.from_pretrained(repo, torch_dtype=dtype).to(device)
    pipe = AnimateDiffPipeline.from_pretrained(base, motion_adapter=adapter, torch_dtype=dtype).to(device)
    if scheduler_type == "DDIM":
        pipe.scheduler = DDIMScheduler.from_pretrained(
            base,
            subfolder="scheduler",
            clip_sample=False,
            timestep_spacing="trailing",
            beta_schedule="linear",
            steps_offset=1,
        )
    elif scheduler_type == "Euler":
        pipe.scheduler = EulerDiscreteScheduler.from_config(
            pipe.scheduler.config,
            clip_sample=False,
            timestep_spacing="trailing",
            beta_schedule="linear",
            steps_offset=1,
        )
    pipe.enable_vae_slicing()
    pipe.enable_model_cpu_offload()
    return pipe

# Initialize Lightning pipeline with EulerDiscreteScheduler
step = 8  # Options: [1,2,4,8]
repo_lightning = "ByteDance/AnimateDiff-Lightning"
ckpt_lightning = f"animatediff_lightning_{step}step_diffusers.safetensors"
lightning_pipe = initialize_pipeline(repo_lightning, ckpt_lightning, base, device, dtype, use_checkpoint=True, scheduler_type="Euler")

# Initialize Original pipeline with DDIMScheduler
repo_original = "guoyww/animatediff-motion-adapter-v1-5-2"
original_pipe = initialize_pipeline(repo_original, None, base, device, dtype, use_checkpoint=False, scheduler_type="DDIM")

# Function to generate GIF
def generate_gif(pipe, prompt, negative_prompt, num_frames, guidance_scale, num_inference_steps, seed, filename):
    output = pipe(
        prompt=prompt,
        negative_prompt=negative_prompt,
        num_frames=num_frames,
        guidance_scale=guidance_scale,
        num_inference_steps=num_inference_steps,
        generator=torch.Generator(device).manual_seed(seed)
    )
    export_to_gif(output.frames[0], filename)
    print(f"{filename} created")
    display(Image(filename))

# Generate GIFs
generate_gif(lightning_pipe, prompt, negative_prompt, num_frames, guidance_scale, num_inference_steps, seed, "lightning_animation.gif")
generate_gif(original_pipe, prompt, negative_prompt, num_frames, guidance_scale, num_inference_steps, seed, "original_animation.gif")


In [ ]:
prompt = "A cozy cabin with snow falling outside."

generate_gif(lightning_pipe, prompt, negative_prompt, num_frames, guidance_scale, num_inference_steps, seed, "lightning_animation.gif")
generate_gif(original_pipe, prompt, negative_prompt, num_frames, guidance_scale, num_inference_steps, seed, "original_animation.gif")


In [ ]:
import json
import os
from tqdm import tqdm

# Define paths
prompts_file = "Text_Prompts.json"
output_dir = "Generated_GIFs"

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Load prompts from JSON file
with open(prompts_file, 'r') as file:
    data = json.load(file)

for theme in data["Themes"]:
    theme_name = theme["Name"].replace(" ", "_")
    theme_dir = os.path.join(output_dir, theme_name)
    os.makedirs(theme_dir, exist_ok=True)
    
    for idx, prompt in enumerate(theme["Prompts"], 1):
        for model_name, pipe in [("lightning_pipe", lightning_pipe), ("original_pipe", original_pipe)]:
            model_dir = os.path.join(theme_dir, model_name)
            os.makedirs(model_dir, exist_ok=True)
            
            filename = f"{theme_name}_{model_name}_Prompt{idx}.gif"
            filepath = os.path.join(model_dir, filename)
            
            generate_gif(
                pipe=pipe,
                prompt=prompt,
                negative_prompt="bad quality, worse quality",  # Update if needed
                num_frames=num_frames,
                guidance_scale=guidance_scale,
                num_inference_steps=num_inference_steps,
                seed=seed,
                filename=filepath,
            )
                